In [1]:
import pandas as pd
import numpy as np
import os
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt 
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
folder_path = '/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/'
dataframes = []
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        dataframes.append(df)
merged_df = pd.concat(dataframes, ignore_index=True)

In [3]:
# action_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/action_df.csv")
# adventure_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/adventure_df.csv")
# animation_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/animation_df.csv")
# biography_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/biography_df.csv")
# comedy_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/comedy_df.csv")
# crime_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/crime_df.csv")
# documentary_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/documentary_df.csv")

# merged_df = pd.concat([action_df, adventure_df, animation_df, biography_df, comedy_df, crime_df, documentary_df])
# merged_df.shape

In [4]:
merged_df.sample(5)

,Unnamed: 0,movie,imdb_id,genre,plot,image_url
1092,92,The Gentlemen,tt8367814,"['Action', ' Comedy', ' Crime']","A talented American graduate of Oxford, using ...",http://img.omdbapi.com/?i=tt8367814&h=600&apik...
4373,60,The Killing Fields,tt0087553,"['Biography', ' Drama', ' History']",Sydney Schanberg is a New York Times journalis...,http://img.omdbapi.com/?i=tt0087553&h=600&apik...
1040,40,Blade Runner,tt0083658,"['Action', ' Drama', ' Sci-Fi']","In the early twenty-first century, the Tyrell ...",http://img.omdbapi.com/?i=tt0083658&h=600&apik...
4672,109,Vera Cruz,tt0047647,"['Adventure', ' Drama', ' Western']","After the American Civil War, mercenaries trav...",http://img.omdbapi.com/?i=tt0047647&h=600&apik...
676,176,Gone Girl,tt2267998,"['Drama', ' Mystery', ' Thriller']",On the occasion of his fifth wedding anniversa...,http://img.omdbapi.com/?i=tt2267998&h=600&apik...


In [5]:
merged_df = merged_df.drop(columns=['Unnamed: 0'])

In [6]:
merged_df = merged_df.drop_duplicates()
merged_df.shape

(2590, 5)

In [7]:
merged_df.sample(5)

,movie,imdb_id,genre,plot,image_url
98,The Muppet Movie,tt0079588,"['Adventure', ' Comedy', ' Family']","While living the quiet life in a swamp, Kermit...",http://img.omdbapi.com/?i=tt0079588&h=600&apik...
2489,The Tale,tt4015500,"['Biography', ' Drama', ' Mystery']",Jennifer Fox faces a host of life-altering que...,http://img.omdbapi.com/?i=tt4015500&h=600&apik...
3538,The Witches,tt0100944,"['Adventure', ' Comedy', ' Family']",A recently orphaned young boy is taken to a se...,http://img.omdbapi.com/?i=tt0100944&h=600&apik...
221,Miracles from Heaven,tt4257926,"['Biography', ' Drama', ' Family']",MIRACLES FROM HEAVEN is based on the incredibl...,http://img.omdbapi.com/?i=tt4257926&h=600&apik...
4722,Meek's Cutoff,tt1518812,"['Adventure', ' Drama', ' Thriller']","The year is 1845, the earliest days of the Ore...",http://img.omdbapi.com/?i=tt1518812&h=600&apik...


In [8]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

In [9]:
def cleaning(sentence):
    
    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase 
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers
    
    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## remove punctuation
    
    tokenized_sentence = word_tokenize(sentence) ## tokenize 
    stop_words = set(stopwords.words('english')) ## define stopwords
    
    tokenized_sentence_cleaned = [ ## remove stopwords
        w for w in tokenized_sentence if not w in stop_words
    ]

    lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "v") 
        for word in tokenized_sentence_cleaned
    ]
    
    cleaned_sentence = ' '.join(word for word in lemmatized)
    
    return cleaned_sentence

In [10]:
merged_df["clean_plot"] = merged_df["plot"].apply(cleaning)

In [11]:
merged_df

,movie,imdb_id,genre,plot,image_url,clean_plot
0,Spider-Man: Across the Spider-Verse,tt9362722,"['Animation', ' Action', ' Adventure']",Miles Morales returns for the next chapter of ...,http://img.omdbapi.com/?i=tt9362722&h=600&apik...,miles morales return next chapter oscar®winnin...
1,It's a Wonderful Life,tt0038650,"['Drama', ' Family', ' Fantasy']",George Bailey has spent his entire life giving...,http://img.omdbapi.com/?i=tt0038650&h=600&apik...,george bailey spend entire life give people be...
2,The Lion King,tt0110357,"['Animation', ' Adventure', ' Drama']",A young lion prince is cast out of his pride b...,http://img.omdbapi.com/?i=tt0110357&h=600&apik...,young lion prince cast pride cruel uncle claim...
3,Spider-Man: Into the Spider-Verse,tt4633694,"['Animation', ' Action', ' Adventure']","Phil Lord and Christopher Miller, the creative...",http://img.omdbapi.com/?i=tt4633694&h=600&apik...,phil lord christopher miller creative mind beh...
4,Coco,tt2380307,"['Animation', ' Adventure', ' Drama']",Despite his family's baffling generations-old ...,http://img.omdbapi.com/?i=tt2380307&h=600&apik...,despite familys baffle generationsold ban musi...
...,...,...,...,...,...,...
5010,Pootie Tang,tt0258038,"['Action', ' Adventure', ' Comedy']","Pootie Tang, the musician/actor/folk hero of t...",http://img.omdbapi.com/?i=tt0258038&h=600&apik...,pootie tang musicianactorfolk hero ghetto chro...
5011,Pinocchio,tt4593060,"['Adventure', ' Comedy', ' Drama']",A live-action adaptation of Disney's 'Pinocchio'.,http://img.omdbapi.com/?i=tt4593060&h=600&apik...,liveaction adaptation disneys pinocchio
5012,Fame,tt1016075,"['Drama', ' Musical', ' Romance']","An updated version of the musical Fame (1980),...",http://img.omdbapi.com/?i=tt1016075&h=600&apik...,update version musical fame center students ne...
5013,High School Musical 3: Senior Year,tt0962726,"['Comedy', ' Drama', ' Family']",Troy and the gang of East High School are goin...,http://img.omdbapi.com/?i=tt0962726&h=600&apik...,troy gang east high school go senior year face...


In [12]:
merged_df["genre"] = (
    merged_df["genre"]
    .apply(eval)
    .apply(lambda x: [genre.strip() for genre in x])
    )

In [13]:
#check the genre we have
merged_df["genre"].explode().value_counts()

Drama        1684
Comedy        806
Adventure     587
Action        478
Crime         395
Biography     378
Romance       344
Animation     287
Thriller      249
Mystery       235
Horror        229
Music         204
History       190
Fantasy       183
Family        179
Sport         168
War           130
Sci-Fi        122
Western       104
Musical       103
Film-Noir      59
Name: genre, dtype: int64

In [14]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(merged_df["genre"])

# transform target variable
y = multilabel_binarizer.transform(merged_df['genre'])

In [15]:
y.shape

(2590, 21)

In [16]:
genre_names = multilabel_binarizer.classes_

# Adding 
for i in range(len(genre_names)):
    merged_df[f"{genre_names[i]}"] = y[:,i]

merged_df.shape

(2590, 27)

In [17]:
merged_df.head(5)

,movie,imdb_id,genre,plot,image_url,clean_plot,Action,Adventure,Animation,Biography,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,Spider-Man: Across the Spider-Verse,tt9362722,"[Animation, Action, Adventure]",Miles Morales returns for the next chapter of ...,http://img.omdbapi.com/?i=tt9362722&h=600&apik...,miles morales return next chapter oscar®winnin...,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,It's a Wonderful Life,tt0038650,"[Drama, Family, Fantasy]",George Bailey has spent his entire life giving...,http://img.omdbapi.com/?i=tt0038650&h=600&apik...,george bailey spend entire life give people be...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Lion King,tt0110357,"[Animation, Adventure, Drama]",A young lion prince is cast out of his pride b...,http://img.omdbapi.com/?i=tt0110357&h=600&apik...,young lion prince cast pride cruel uncle claim...,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Spider-Man: Into the Spider-Verse,tt4633694,"[Animation, Action, Adventure]","Phil Lord and Christopher Miller, the creative...",http://img.omdbapi.com/?i=tt4633694&h=600&apik...,phil lord christopher miller creative mind beh...,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Coco,tt2380307,"[Animation, Adventure, Drama]",Despite his family's baffling generations-old ...,http://img.omdbapi.com/?i=tt2380307&h=600&apik...,despite familys baffle generationsold ban musi...,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
X_train, X_test, y_train, y_test = train_test_split(merged_df['clean_plot'], y, test_size=0.3)

In [19]:
# Instantiating the TfidfVectorizer
tf_idf_vectorizer = TfidfVectorizer(max_df=0.8, min_df=0.05, max_features=10000)

# Training it on the texts
X_train_vec = pd.DataFrame(tf_idf_vectorizer.fit_transform(X_train).toarray(),
                 columns = tf_idf_vectorizer.get_feature_names_out())
X_test_vec =  pd.DataFrame(tf_idf_vectorizer.transform(X_test).toarray(),
                 columns = tf_idf_vectorizer.get_feature_names_out())

In [20]:
X_train_vec.head()

,along,also,american,another,away,back,base,become,begin,believe,...,war,way,well,wife,woman,work,world,years,york,young
0,0.0,0.000000,0.0,0.000000,0.0,0.1683,0.0,0.141623,0.0,0.000000,...,0.0,0.164736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.143164,0.0,0.168513,0.0,0.0000,0.0,0.000000,0.0,0.160623,...,0.0,0.133344,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.397506


In [21]:
X_test_vec.head()

,along,also,american,another,away,back,base,become,begin,believe,...,war,way,well,wife,woman,work,world,years,york,young
0,0.0,0.0,0.0,0.236267,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.257461,0.0,0.000000,0.0,...,0.0000,0.214238,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.354714,0.0,...,0.0000,0.000000,0.0,0.376944,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.4067,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score

In [23]:
log = LogisticRegression()
model_test_1 = OneVsRestClassifier(log)

model_test_1.fit(X_train_vec, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [24]:
y_pred = model_test_1.predict(X_test_vec)

In [25]:
f1_score(y_test, y_pred, average="micro")

0.388336646785951

In [26]:
multilabel_binarizer.inverse_transform(y_pred)[1]

('Drama',)

In [27]:
merged_df[merged_df.clean_plot == X_test.iat[1]]

,movie,imdb_id,genre,plot,image_url,clean_plot,Action,Adventure,Animation,Biography,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
3457,Cemetery Man,tt0109592,"[Comedy, Horror]",This movie is based on a novel of Tiziano Scla...,http://img.omdbapi.com/?i=tt0109592&h=600&apik...,movie base novel tiziano sclavi always reflect...,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

# Instantiate the LDA 
n_components = 15
lda_model = LatentDirichletAllocation(n_components=n_components, max_iter = 1000)

# Fit the LDA on the vectorized documents
lda_model.fit(X_train_vec)

In [ ]:
X_train_topic_mixture = lda_model.transform(X_train_vec)
X_train_topic_df = pd.DataFrame(X_train_topic_mixture)
X_train_topic_df

In [ ]:
X_test_topic_mixture = lda_model.transform(X_test_vec)
X_test_topic_df = pd.DataFrame(X_test_topic_mixture)
X_test_topic_df

In [ ]:
model_test_2 = OneVsRestClassifier(log)
model_test_2.fit(X_train_topic_df, y_train)

In [ ]:
y_pred_2 = model_test_2.predict(X_test_topic_df)
f1_score(y_test, y_pred, average="micro")

In [ ]:
multilabel_binarizer.inverse_transform(y_pred_2)[0:10]

In [ ]:
multilabel_binarizer.inverse_transform(y_test)[0:10]

In [ ]:
model_test_3 = OneVsRestClassifier(log)
model_test_3.fit(X_train_topic_df, y_train)
model_test_3.fit(X_train_vec, y_train)

In [ ]:
y_pred_3 = model_test_2.predict(X_test_topic_df)
f1_score(y_test, y_pred, average="micro")